# Energy Data Analytics - Delta Lake Queries

This notebook demonstrates reading Delta format data and executing three core energy analytics queries using Spark SQL:

1. **Daily Production Trends** - Daily electricity production by production type
2. **Underperformance Prediction Features** - ML features for energy production forecasting
3. **Wind Price Analysis** - Wind power production vs electricity prices

## Data Sources
- Delta tables in the Gold layer: `gold_fact_power`, `gold_dim_production_type`, `gold_fact_power_30min_agg`

## 1. Setup Spark Session and Delta Lake

In [1]:
# Import required libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# Create Spark session with Delta Lake configuration
spark = (SparkSession.builder.master("local[*]")
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.0.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore")
    .config("spark.driver.memory", "4g")  # Increase to 4GB
    .config("spark.driver.maxResultSize", "2g")
    .config("spark.sql.shuffle.partitions", "200")  # Reduce shuffle partitions
    .config("spark.sql.autoBroadcastJoinThreshold", "10485760")  # 10MB
    .config("spark.executor.memory", "4g")  # If using executors
    .getOrCreate())

print("Spark Session created with Delta Lake support")
print(f"Spark Version: {spark.version}")
print(f"Application Name: {spark.sparkContext.appName}")

:: loading settings :: url = jar:file:/usr/local/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/sreejadath/.ivy2/cache
The jars for the packages stored in: /Users/sreejadath/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4ef33f5c-d276-4c8a-ba57-360b444b806d;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.0.0 in central
	found io.delta#delta-storage;2.0.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 121ms :: artifacts dl 5ms
	:: modules in use:
	io.delta#delta-core_2.12;2.0.0 from central in [default]
	io.delta#delta-storage;2.0.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|d

Spark Session created with Delta Lake support
Spark Version: 3.5.4
Application Name: pyspark-shell


## 2. Read Delta Lake Tables

In [2]:
# Define Delta table paths (update these paths to match your environment)
delta_base_path = "/Users/srsu/Downloads/spark_datapipeline/delta_lake"

# Read Delta tables from the Gold layer
print("📊 Reading Delta tables...")

try:
    # Read dimension table
    gold_dim_production_type = spark.read.format("delta").load(f"{delta_base_path}/gold/gold_dim_production_type")
    gold_dim_production_type.createOrReplaceTempView("gold_dim_production_type")
    print(f"Loaded gold_dim_production_type: {gold_dim_production_type.count()} records")
    
    # Read daily fact table
    gold_fact_power = spark.read.format("delta").load(f"{delta_base_path}/gold/gold_fact_power")
    gold_fact_power.createOrReplaceTempView("gold_fact_power")
    print(f"Loaded gold_fact_power: {gold_fact_power.count()} records")
    
    # Read 30-minute aggregated fact table
    gold_fact_power_30min_agg = spark.read.format("delta").load(f"{delta_base_path}/gold/gold_fact_power_30min_agg")
    gold_fact_power_30min_agg.createOrReplaceTempView("gold_fact_power_30min_agg")
    print(f"Loaded gold_fact_power_30min_agg: {gold_fact_power_30min_agg.count()} records")
    
except Exception as e:
    print(f"Could not read Delta tables: {e}")
    print("Creating sample data for POC demonstration...")
    

📊 Reading Delta tables...
Could not read Delta tables: An error occurred while calling o67.load.
: com.google.common.util.concurrent.ExecutionError: java.lang.NoSuchMethodError: 'java.lang.String org.apache.spark.ErrorInfo.messageFormat()'
	at com.google.common.cache.LocalCache$Segment.get(LocalCache.java:2261)
	at com.google.common.cache.LocalCache.get(LocalCache.java:4000)
	at com.google.common.cache.LocalCache$LocalManualCache.get(LocalCache.java:4789)
	at org.apache.spark.sql.delta.DeltaLog$.getDeltaLogFromCache$1(DeltaLog.scala:606)
	at org.apache.spark.sql.delta.DeltaLog$.apply(DeltaLog.scala:613)
	at org.apache.spark.sql.delta.DeltaLog$.forTable(DeltaLog.scala:511)
	at org.apache.spark.sql.delta.catalog.DeltaTableV2.deltaLog$lzycompute(DeltaTableV2.scala:85)
	at org.apache.spark.sql.delta.catalog.DeltaTableV2.deltaLog(DeltaTableV2.scala:85)
	at org.apache.spark.sql.delta.catalog.DeltaTableV2.$anonfun$snapshot$3(DeltaTableV2.scala:114)
	at scala.Option.getOrElse(Option.scala:189)

## 3. Query 1: Daily Production Trends

This query analyzes daily electricity production trends by production type.

In [3]:
# Query 1: Daily Production Trends
daily_production_query = """
SELECT
  f.year,
  f.month,
  f.day,
  d.production_plant_name AS production_type,
  SUM(f.electricity_produced) AS total_daily_production
FROM gold_fact_power f
JOIN gold_dim_production_type d ON f.production_type_id = d.production_type_id
WHERE f.country = 'de'
GROUP BY f.year, f.month, f.day, d.production_plant_name
ORDER BY f.year, f.month, f.day, d.production_plant_name
"""

print("🔍 Executing Query 1: Daily Production Trends")
print("=" * 50)

try:
    daily_trends_df = spark.sql(daily_production_query)
    
    print(f"Query executed successfully!")
    print(f"Results: {daily_trends_df.count()} records found")
    
    print("\nSample Results:")
    daily_trends_df.show(10, truncate=False)
    
    # Show schema
    print("\nSchema:")
    daily_trends_df.printSchema()
    
except Exception as e:
    print(f"Query failed: {e}")
    print("This is expected if Delta tables don't exist - using sample data for POC")

🔍 Executing Query 1: Daily Production Trends
Query failed: An error occurred while calling o44.sql.
: java.lang.NoSuchMethodError: 'java.lang.Object org.apache.spark.sql.catalyst.parser.ParserUtils$.withOrigin(org.antlr.v4.runtime.ParserRuleContext, scala.Function0)'
	at io.delta.sql.parser.DeltaSqlAstBuilder.visitSingleStatement(DeltaSqlParser.scala:244)
	at io.delta.sql.parser.DeltaSqlAstBuilder.visitSingleStatement(DeltaSqlParser.scala:146)
	at io.delta.sql.parser.DeltaSqlBaseParser$SingleStatementContext.accept(DeltaSqlBaseParser.java:165)
	at org.antlr.v4.runtime.tree.AbstractParseTreeVisitor.visit(AbstractParseTreeVisitor.java:18)
	at io.delta.sql.parser.DeltaSqlParser.$anonfun$parsePlan$1(DeltaSqlParser.scala:74)
	at io.delta.sql.parser.DeltaSqlParser.parse(DeltaSqlParser.scala:103)
	at io.delta.sql.parser.DeltaSqlParser.parsePlan(DeltaSqlParser.scala:73)
	at org.apache.spark.sql.SparkSession.$anonfun$sql$5(SparkSession.scala:684)
	at org.apache.spark.sql.catalyst.QueryPlanningT

ANTLR Tool version 4.8 used for code generation does not match the current runtime version 4.9.3
ANTLR Runtime version 4.8 used for parser compilation does not match the current runtime version 4.9.3
ANTLR Tool version 4.8 used for code generation does not match the current runtime version 4.9.3
ANTLR Runtime version 4.8 used for parser compilation does not match the current runtime version 4.9.3


## 4. Query 2: Underperformance Prediction Features

This query generates ML features for predicting energy production underperformance with lag features and rolling averages.

In [4]:
# Query 2: Underperformance Prediction Features
underperformance_query = """
SELECT
    f.timestamp_30min,
    f.production_type_id,
    d.production_plant_name,
    d.energy_category,
    d.controllability_type,
    f.total_electricity_produced,
    f.year, f.month, f.day, f.hour, f.minute_interval_30,
    LAG(f.total_electricity_produced, 48) OVER (PARTITION BY f.production_type_id ORDER BY f.timestamp_30min) AS lag_1d,
    LAG(f.total_electricity_produced, 336) OVER (PARTITION BY f.production_type_id ORDER BY f.timestamp_30min) AS lag_1w,
    AVG(f.total_electricity_produced) OVER (
        PARTITION BY f.production_type_id, f.hour, f.minute_interval_30
        ORDER BY f.timestamp_30min
        RANGE BETWEEN 336 PRECEDING AND 1 PRECEDING
    ) AS rolling_7d_avg
FROM gold_fact_power_30min_agg f
JOIN gold_dim_production_type d ON f.production_type_id = d.production_type_id
WHERE f.country = 'de' AND d.active_flag = TRUE
"""

print("🔍 Executing Query 2: ML Features for Underperformance Prediction")
print("=" * 65)

try:
    underperformance_query_df = spark.sql(underperformance_query)
    
    print(f"Query executed successfully!")
    print(f"Results: {underperformance_query_df.count()} records found")
    
    print("\nSample ML Features:")
    underperformance_query_df.show(5, truncate=False)
    
    # Show feature statistics
    print("\nFeature Statistics:")
    underperformance_query_df.select("total_electricity_produced", "lag_1d", "lag_1w", "rolling_7d_avg").describe().show()
    
except Exception as e:
    print(f" Query failed: {e}")
    print("This is expected if 30-minute Delta table doesn't exist - using sample data for POC")

🔍 Executing Query 2: ML Features for Underperformance Prediction
 Query failed: An error occurred while calling o44.sql.
: java.lang.NoSuchMethodError: 'java.lang.Object org.apache.spark.sql.catalyst.parser.ParserUtils$.withOrigin(org.antlr.v4.runtime.ParserRuleContext, scala.Function0)'
	at io.delta.sql.parser.DeltaSqlAstBuilder.visitSingleStatement(DeltaSqlParser.scala:244)
	at io.delta.sql.parser.DeltaSqlAstBuilder.visitSingleStatement(DeltaSqlParser.scala:146)
	at io.delta.sql.parser.DeltaSqlBaseParser$SingleStatementContext.accept(DeltaSqlBaseParser.java:165)
	at org.antlr.v4.runtime.tree.AbstractParseTreeVisitor.visit(AbstractParseTreeVisitor.java:18)
	at io.delta.sql.parser.DeltaSqlParser.$anonfun$parsePlan$1(DeltaSqlParser.scala:74)
	at io.delta.sql.parser.DeltaSqlParser.parse(DeltaSqlParser.scala:103)
	at io.delta.sql.parser.DeltaSqlParser.parsePlan(DeltaSqlParser.scala:73)
	at org.apache.spark.sql.SparkSession.$anonfun$sql$5(SparkSession.scala:684)
	at org.apache.spark.sql.ca

## 5. Query 3: Wind Price Analysis

This query analyzes the relationship between wind power production (offshore and onshore) and electricity prices.

In [5]:
# Query 3: Wind Price Analysis
wind_price_query = """
SELECT
  f.year, f.month, f.day,
  d.production_plant_name AS production_type,
  SUM(f.electricity_produced) AS total_daily_production_mw,
  AVG(f.electricity_price) AS avg_daily_price_eur_per_mwh
FROM gold_fact_power f
JOIN gold_dim_production_type d ON f.production_type_id = d.production_type_id
WHERE f.country = 'de'
  AND d.production_plant_name IN ('Wind_Offshore', 'Wind_Onshore') 
  AND d.active_flag = TRUE 
GROUP BY f.year, f.month, f.day, d.production_plant_name
ORDER BY f.year, f.month, f.day, d.production_plant_name
"""

print("🔍 Executing Query 3: Wind Power vs Price Analysis")
print("=" * 50)

try:
    wind_analysis_df = spark.sql(wind_price_query)
    
    print(f"Query executed successfully!")
    print(f"Results: {wind_analysis_df.count()} records found")
    
    print("\nWind Power vs Price Results:")
    wind_analysis_df.show(10, truncate=False)
    
    # Summary statistics by wind type
    if wind_analysis_df.count() > 0:
        print("\nSummary by Wind Type:")
        wind_summary = wind_analysis_df.groupBy("production_type").agg(
            avg("total_daily_production_mw").alias("avg_production"),
            avg("avg_daily_price_eur_per_mwh").alias("avg_price"),
            count("*").alias("total_days")
        )
        wind_summary.show(truncate=False)
    
except Exception as e:
    print(f"Query failed: {e}")
    print("This is expected if Delta tables don't exist - using sample data for POC")

🔍 Executing Query 3: Wind Power vs Price Analysis
Query failed: An error occurred while calling o44.sql.
: java.lang.NoSuchMethodError: 'java.lang.Object org.apache.spark.sql.catalyst.parser.ParserUtils$.withOrigin(org.antlr.v4.runtime.ParserRuleContext, scala.Function0)'
	at io.delta.sql.parser.DeltaSqlAstBuilder.visitSingleStatement(DeltaSqlParser.scala:244)
	at io.delta.sql.parser.DeltaSqlAstBuilder.visitSingleStatement(DeltaSqlParser.scala:146)
	at io.delta.sql.parser.DeltaSqlBaseParser$SingleStatementContext.accept(DeltaSqlBaseParser.java:165)
	at org.antlr.v4.runtime.tree.AbstractParseTreeVisitor.visit(AbstractParseTreeVisitor.java:18)
	at io.delta.sql.parser.DeltaSqlParser.$anonfun$parsePlan$1(DeltaSqlParser.scala:74)
	at io.delta.sql.parser.DeltaSqlParser.parse(DeltaSqlParser.scala:103)
	at io.delta.sql.parser.DeltaSqlParser.parsePlan(DeltaSqlParser.scala:73)
	at org.apache.spark.sql.SparkSession.$anonfun$sql$5(SparkSession.scala:684)
	at org.apache.spark.sql.catalyst.QueryPlan

In [6]:
#Stop Spark session
spark.stop()